In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from datetime import datetime
import random
import os
import sys
import pickle

In [2]:
tournaments_df = pd.read_csv('../Data/tournaments.csv').set_index(['Name', 'Year'])
players_df = pd.read_csv('../Data/players.csv').set_index('Name')

matches_df = pd.read_csv('../Data/matches.csv')
matches_df = matches_df.drop(matches_df.columns[[0]], axis = 1)
df = matches_df.drop(['Tournament Name', 'Year', 'Date', 'Surface', 'Score'], axis = 1)
df

,Player 1,Player 2,Winner,Player 1 Ranking,Player 2 Ranking,Player 1 Previous Wins,Player 2 Previous Wins,P1 Surface Wins,P1 Surface Matches,P2 Surface Wins,P2 Surface Matches,P1 Last 10 Matches,P2 Last 10 Matches
0,Lleyton Hewitt,Guillermo Canas,Lleyton Hewitt,6.0,61.0,1.0,0.0,10.0,10.0,4.0,5.0,8.0,7.0
1,Lleyton Hewitt,Roger Federer,Lleyton Hewitt,6.0,15.0,0.0,0.0,10.0,10.0,5.0,7.0,8.0,7.0
2,Guillermo Canas,Tommy Robredo,Guillermo Canas,61.0,69.0,1.0,0.0,4.0,5.0,3.0,4.0,7.0,6.0
3,Lleyton Hewitt,Gilles Elseneer,Lleyton Hewitt,6.0,323.0,0.0,0.0,10.0,10.0,4.0,6.0,8.0,4.0
4,Roger Federer,Raemon Sluiter,Roger Federer,15.0,107.0,0.0,0.0,5.0,7.0,4.0,6.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86551,Tatsuma Ito,Thomas Fabbiano,Tatsuma Ito,141.0,88.0,0.0,0.0,75.0,159.0,57.0,126.0,7.0,4.0
86552,Brayden Schnur,Goncalo Oliveira,Brayden Schnur,95.0,257.0,0.0,0.0,15.0,33.0,0.0,4.0,1.0,4.0
86553,Henri Laaksonen,Runhao Hua,Henri Laaksonen,110.0,685.0,0.0,0.0,24.0,60.0,0.0,2.0,4.0,0.0
86554,Yuichi Sugita,Zhao Zhao,Yuichi Sugita,129.0,1602.0,0.0,0.0,86.0,177.0,0.0,1.0,7.0,0.0


In [3]:
p1, p2, p1_rank, p2_rank = list(df['Player 1']), list(df['Player 2']), list(df['Player 1 Ranking']), list(df['Player 2 Ranking'])
p1_prev, p2_prev = list(df['Player 1 Previous Wins']), list(df['Player 2 Previous Wins'])
p1_surface, p2_surface = list(df['P1 Surface Matches']), list(df['P2 Surface Matches'])
p1_surf_wins, p2_surf_wins = list(df['P1 Surface Wins']), list(df['P2 Surface Wins'])
p1_10, p2_10 = list(df['P1 Last 10 Matches']), list(df['P2 Last 10 Matches'])

for i in range(len(p1)):
    if random.random() > 0.5:
        temp = p1[i]
        p1[i] = p2[i]
        p2[i] = temp

        temp = p1_rank[i]
        p1_rank[i] = p2_rank[i]
        p2_rank[i] = temp
        
        temp = p1_prev[i]
        p1_prev[i] = p2_prev[i]
        p2_prev[i] = temp

        temp = p1_surface[i]
        p1_surface[i] = p2_surface[i]
        p2_surface[i] = temp

        temp = p1_surf_wins[i]
        p1_surf_wins[i] = p2_surf_wins[i]
        p2_surf_wins[i] = temp

        temp = p1_10[i]
        p1_10[i] = p2_10[i]
        p2_10[i] = temp        
        
df = pd.DataFrame({'P1' : p1, 'P2' : p2, 'Winner' : df['Winner'], 'P1 Ranking' : p1_rank, 'P2 Ranking' : p2_rank})
df['P1 Previous Wins'], df['P2 Previous Wins'], df['P1 Surface Matches'], df['P2 Surface Matches'] = p1_prev, p2_prev, p1_surface, p2_surface
df['P1 Surface Wins'], df['P2 Surface Wins'], df['P1 Last 10 Matches'], df['P2 Last 10 Matches'] = p1_surf_wins, p2_surf_wins, p1_10, p2_10
df['winner'] = df.apply(lambda x: 0 if x['Winner'] == x['P1'] else 1, axis = 1)

In [4]:
df

,P1,P2,Winner,P1 Ranking,P2 Ranking,P1 Previous Wins,P2 Previous Wins,P1 Surface Matches,P2 Surface Matches,P1 Surface Wins,P2 Surface Wins,P1 Last 10 Matches,P2 Last 10 Matches,winner
0,Guillermo Canas,Lleyton Hewitt,Lleyton Hewitt,61.0,6.0,0.0,1.0,5.0,10.0,4.0,10.0,7.0,8.0,1
1,Roger Federer,Lleyton Hewitt,Lleyton Hewitt,15.0,6.0,0.0,0.0,7.0,10.0,5.0,10.0,7.0,8.0,1
2,Tommy Robredo,Guillermo Canas,Guillermo Canas,69.0,61.0,0.0,1.0,4.0,5.0,3.0,4.0,6.0,7.0,1
3,Lleyton Hewitt,Gilles Elseneer,Lleyton Hewitt,6.0,323.0,0.0,0.0,10.0,6.0,10.0,4.0,8.0,4.0,0
4,Raemon Sluiter,Roger Federer,Roger Federer,107.0,15.0,0.0,0.0,6.0,7.0,4.0,5.0,3.0,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86551,Thomas Fabbiano,Tatsuma Ito,Tatsuma Ito,88.0,141.0,0.0,0.0,126.0,159.0,57.0,75.0,4.0,7.0,1
86552,Brayden Schnur,Goncalo Oliveira,Brayden Schnur,95.0,257.0,0.0,0.0,33.0,4.0,15.0,0.0,1.0,4.0,0
86553,Henri Laaksonen,Runhao Hua,Henri Laaksonen,110.0,685.0,0.0,0.0,60.0,2.0,24.0,0.0,4.0,0.0,0
86554,Zhao Zhao,Yuichi Sugita,Yuichi Sugita,1602.0,129.0,0.0,0.0,1.0,177.0,0.0,86.0,0.0,7.0,1


In [5]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [6]:
y = df['winner']
X = df.drop(['P1', 'P2', 'Winner', 'winner'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

,P1 Ranking,P2 Ranking,P1 Previous Wins,P2 Previous Wins,P1 Surface Matches,P2 Surface Matches,P1 Surface Wins,P2 Surface Wins,P1 Last 10 Matches,P2 Last 10 Matches
35820,152.0,946.0,0.0,0.0,18.0,3.0,11.0,1.0,3.0,5.0
34404,147.0,899.0,0.0,0.0,20.0,12.0,11.0,5.0,4.0,4.0
41328,139.0,663.0,0.0,0.0,5.0,2.0,3.0,1.0,1.0,1.0
7343,165.0,212.0,0.0,0.0,48.0,9.0,20.0,4.0,4.0,4.0
34888,84.0,107.0,0.0,0.0,2.0,3.0,1.0,2.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...
50090,87.0,97.0,0.0,0.0,86.0,49.0,36.0,24.0,6.0,4.0
84856,184.0,0.0,0.0,0.0,5.0,7.0,2.0,2.0,5.0,4.0
22076,1031.0,119.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,1.0
49307,45.0,13.0,0.0,0.0,19.0,145.0,9.0,76.0,3.0,6.0


In [7]:
scaler = StandardScaler()
scaler.fit(X_train)
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
X_train = scaler.transform(X_train)
X_train

array([[ 0.09160804,  4.12268237, -0.37210746, ..., -0.70852025,
        -0.8136812 ,  0.22307406],
       [ 0.06675752,  3.88431864, -0.37210746, ..., -0.65305865,
        -0.29430894, -0.2954054 ],
       [ 0.02699671,  2.68742839, -0.37210746, ..., -0.70852025,
        -1.85242571, -1.85084378],
       ...,
       [ 4.46032803, -0.07150506, -0.37210746, ..., -0.70852025,
        -2.37179796, -1.85084378],
       [-0.44019292, -0.60909136, -0.37210746, ...,  0.33138474,
        -0.8136812 ,  0.74155352],
       [-0.47498364, -0.65473548, -0.37210746, ..., -0.12617346,
         0.22506331,  0.74155352]])

In [8]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

c:\Users\aman0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


0.6252888170055453

In [9]:
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

c:\Users\aman0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\aman0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but LinearSVC was fitted without feature names
  warnings.warn(


0.6311806839186691

In [10]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

c:\Users\aman0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.6240757855822551

In [11]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear SVM classifier (you can adjust hyperparameters as needed)
base_svm = LinearSVC()

# Create a BaggingClassifier with linear SVM as the base estimator
n_estimators = 2  # Number of linear SVM models in the ensemble
ensemble = BaggingClassifier(base_estimator=base_svm, n_estimators=n_estimators, random_state=42)

# Fit the ensemble model on the training data
ensemble.fit(X_train, y_train)

# Make predictions using the ensemble
y_pred = ensemble.predict(X_test)

# Evaluate the accuracy of the ensemble
accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble Accuracy: {accuracy}')


c:\Users\aman0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\aman0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Ensemble Accuracy: 0.5646372458410351


c:\Users\aman0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [12]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create a base classifier (you can replace it with other classifiers)
base_classifier = LogisticRegression()  # Example base classifier (a weak learner)

# Create an AdaBoostClassifier with the base classifier
n_estimators = 50  # Number of boosting iterations
adaboost = AdaBoostClassifier(base_classifier, n_estimators=n_estimators, random_state=42)

# Fit the AdaBoost ensemble model on the training data
adaboost.fit(X_train, y_train)

# Make predictions using the AdaBoost ensemble
y_pred = adaboost.predict(X_test)

# Evaluate the accuracy of the ensemble
accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble Accuracy: {accuracy}')


Ensemble Accuracy: 0.6512823475046211


In [13]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 1)
tree.fit(X_train, y_train)
tree.score(X_test, y_test)

0.5460951940850277

In [14]:
import xgboost as xgb
model = xgb.XGBClassifier(booster = 'dart', n_estimators = 20, max_depth = 10)
model.fit(X_train, y_train)
model.score(X_test, y_test)
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [15]:
model.score(X_test, y_test)

0.7050023105360443